In [1]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
import pandas as pd

df_syf = pd.read_csv("/Users/samar/Desktop/Datathon/consumer_spending_ml/forecast_data/syf_id_20250325.csv")
df_transaction = pd.read_csv("/Users/samar/Desktop/Datathon/consumer_spending_ml/forecast_data/transaction_fact_20250325.csv")
df_accountdim = pd.read_csv("/Users/samar/Desktop/Datathon/consumer_spending_ml/forecast_data/account_dim_20250325.csv")

In [35]:

df_transaction.head()

,current_account_nbr,transaction_type,transaction_code,transaction_date,transaction_amt,payment_type,transaction_return_cnt,transaction_sale_cnt,product_amt,product_qty,...,first_purchase_ind,adj_orgn_tran_dt,curr_markup_fee,fcr_amount,fcr_flag,fcr_rate_of_exchange,frgn_curr_code,frgn_tran_amt,us_equiv_amt,posting_date
0,X7jfKh6xrPAB8Tx6,SALE,253,2024-06-05,15.78,NaN,0,1,NaN,NaN,...,N,NaN,0.0,NaN,NaN,NaN,840,15.78,0.0,2024-06-06
1,yntD77AZDylS48Q4,SALE,253,2024-06-19,14.85,NaN,0,1,NaN,NaN,...,N,NaN,0.0,NaN,NaN,NaN,840,14.85,0.0,2024-06-20
2,LIJPI0sK28Pa7fX2,SALE,253,2024-06-26,136.16,NaN,0,1,NaN,NaN,...,N,NaN,0.0,NaN,NaN,NaN,840,136.16,0.0,2024-06-27
3,CMAr5Apxwdzpvoze,SALE,253,2024-08-15,8.74,NaN,0,1,NaN,NaN,...,N,NaN,0.0,NaN,NaN,NaN,840,8.74,0.0,2024-08-16
4,eJSfTCGPvJulGzd3,SALE,253,2024-08-17,26.65,NaN,0,1,NaN,NaN,...,N,NaN,0.0,NaN,NaN,NaN,840,26.65,0.0,2024-08-18


In [44]:
#build potential features from transaction fact
features_trans = ["transaction_amt", "transaction_date", "current_account_nbr", "frgn_tran_amt", "transaction_type"]
df_target = df_transaction[features_trans]

features_acc = ["payment_hist_1_12_mths","current_account_nbr"]
df_target2 = df_accountdim[features_acc]

features_syf = ["confidence_level", "account_nbr_pty"]
df_target3 = df_syf[features_syf]
df_target3 = df_target3.rename(columns={"account_nbr_pty" : "current_account_nbr"})

df_target.head()



,transaction_amt,transaction_date,current_account_nbr,frgn_tran_amt,transaction_type
0,15.78,2024-06-05,X7jfKh6xrPAB8Tx6,15.78,SALE
1,14.85,2024-06-19,yntD77AZDylS48Q4,14.85,SALE
2,136.16,2024-06-26,LIJPI0sK28Pa7fX2,136.16,SALE
3,8.74,2024-08-15,CMAr5Apxwdzpvoze,8.74,SALE
4,26.65,2024-08-17,eJSfTCGPvJulGzd3,26.65,SALE


In [52]:
df_merged = df_target.copy()

df_merged.loc[:, 'transaction_date'] = pd.to_datetime(df_merged['transaction_date'])
df_merged.loc[:, 'transaction_month'] = df_merged['transaction_date'].dt.month



# Filter transactions that happened in Q4 of the current year
df_q4 = df_transaction[
    (df_transaction['transaction_month'].isin([10, 11, 12]))
]

# Group by customer (current_account_nbr) and sum transaction amounts for Q4
df_q4_spending = df_q4.groupby('current_account_nbr')[['transaction_amt', 'frgn_tran_amt']].sum().reset_index()

# Rename the column to represent Q4 customer spending
df_q4_spending.rename(columns={'transaction_amt': 'customer_spending_q4'}, inplace=True)




# Merge this customer spending target with the features dataset
df_features_target = pd.merge(df_merged, df_q4_spending, on='current_account_nbr', how='left')

cols = ["current_account_nbr", "transaction_type", "transaction_date", "transaction_amt", "transaction_month", "customer_spending_q4"]
df_features_target = df_features_target[cols]

master_ = df_features_target.merge(df_target2, on="current_account_nbr")

master_ = master_[master_["transaction_type"] == "SALE"]
master_['customer_spending_q4'].fillna(0, inplace=True)

master_


,current_account_nbr,transaction_type,transaction_date,transaction_amt,transaction_month,customer_spending_q4,payment_hist_1_12_mths
0,X7jfKh6xrPAB8Tx6,SALE,2024-06-05,15.78,6,4129.69,QQQQQQQQQQQQ
2,X7jfKh6xrPAB8Tx6,SALE,2024-10-09,3.77,10,4129.69,QQQQQQQQQQQQ
4,X7jfKh6xrPAB8Tx6,SALE,2024-11-21,114.04,11,4129.69,QQQQQQQQQQQQ
5,X7jfKh6xrPAB8Tx6,SALE,2024-05-20,26.58,5,4129.69,QQQQQQQQQQQQ
6,X7jfKh6xrPAB8Tx6,SALE,2024-09-03,41.34,9,4129.69,QQQQQQQQQQQQ
...,...,...,...,...,...,...,...
493317,t31snreXki3WZ9v0,SALE,2024-05-03,37.54,5,0.00,ZZZZZZZZZZA%
493321,Ze6quQ9WWmCRtJ59,SALE,2025-02-22,3808.12,2,0.00,AZ
493331,Z9xrNX0MRAwXwVmV,SALE,2025-03-10,327.37,3,0.00,A
493332,4MyzvHSyKNy8fBsC,SALE,2025-03-20,2197.56,3,0.00,"\\\\\\"""""
